In [1]:
import os

In [2]:
import pandas 

In [3]:
import numpy as np

In [4]:
from shutil import copytree

In [5]:
from pyiron import Project

In [6]:
def getdir(path): 
    path_base_name = os.path.basename(path)
    if path_base_name == "":
        return os.path.basename(os.path.dirname(path))
    else: 
        return path_base_name

In [7]:
def update_id_lst(record_lst, job_id_lst):
    masterid_lst = []
    for masterid in record_lst:
        if np.isnan(masterid):
            masterid_lst.append(masterid)
        elif isinstance(masterid, int) or isinstance(masterid, float):
            masterid = int(masterid)
            masterid_lst.append(job_id_lst[masterid])
    return masterid_lst

In [8]:
def import_jobs(directory_to_import_to, archive_directory, df):
    # Copy HDF5 files
    archive_name = getdir(path=archive_directory)
    copytree(archive_directory, directory_to_import_to) 
    
    # Update Database
    pr_import = Project(directory_to_import_to)
    df["project"] = [os.path.join(pr_import.project_path, os.path.relpath(p, archive_name)) for p in df["project"].values]
    df['projectpath'] = len(df) * [pr_import.root_path]
    
    # Add jobs to database 
    job_id_lst = []
    for entry in df.to_dict(orient="records"):
        del entry['id']
        del entry['parentid']
        del entry['masterid']
        entry["timestart"] = pandas.to_datetime(entry["timestart"])
        entry["timestop"] = pandas.to_datetime(entry["timestop"])
        job_id = pr_import.db.add_item_dict(par_dict=entry)
        job_id_lst.append(job_id)
        
    # Update parent and master ids 
    for job_id, masterid, parentid in zip(job_id_lst, update_id_lst(record_lst=df['masterid'].values, job_id_lst=job_id_lst), update_id_lst(record_lst=df['parentid'].values, job_id_lst=job_id_lst)):
        if not np.isnan(masterid) or not np.isnan(parentid):
            pr_import.db.item_update(item_id=job_id, par_dict={'parentid': parentid, 'masterid': masterid})

In [9]:
df = pandas.read_csv("export.csv", index_col=0)
df

,id,status,chemicalformula,job,subjob,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,0,finished,Al,murn,/murn,archive/murn,2020-11-10 11:14:21.689916,2020-11-10 11:14:45.113006,23.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Murnaghan,0.3.0,NaN,NaN
1,1,finished,Al,strain_0_9,/strain_0_9,archive/murn/murn_hdf5,2020-11-10 11:14:22.912227,2020-11-10 11:14:23.365173,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
2,2,finished,Al,strain_0_92,/strain_0_92,archive/murn/murn_hdf5,2020-11-10 11:14:24.927981,2020-11-10 11:14:25.394044,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
3,3,finished,Al,strain_0_94,/strain_0_94,archive/murn/murn_hdf5,2020-11-10 11:14:26.931778,2020-11-10 11:14:27.421954,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
4,4,finished,Al,strain_0_96,/strain_0_96,archive/murn/murn_hdf5,2020-11-10 11:14:29.007742,2020-11-10 11:14:29.431261,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
5,5,finished,Al,strain_0_98,/strain_0_98,archive/murn/murn_hdf5,2020-11-10 11:14:30.930220,2020-11-10 11:14:31.406885,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
6,6,finished,Al,strain_1_0,/strain_1_0,archive/murn/murn_hdf5,2020-11-10 11:14:32.937797,2020-11-10 11:14:33.441668,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
7,7,finished,Al,strain_1_02,/strain_1_02,archive/murn/murn_hdf5,2020-11-10 11:14:34.940175,2020-11-10 11:14:35.461496,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
8,8,finished,Al,strain_1_04,/strain_1_04,archive/murn/murn_hdf5,2020-11-10 11:14:37.038421,2020-11-10 11:14:37.536668,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0
9,9,finished,Al,strain_1_06,/strain_1_06,archive/murn/murn_hdf5,2020-11-10 11:14:39.139254,2020-11-10 11:14:39.683094,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-...,Lammps,0.1,NaN,0.0


In [10]:
directory_to_import_to = os.path.abspath(os.path.join(os.curdir, "new")) 

In [11]:
archive_directory = os.path.abspath(os.path.join(os.curdir, "archive")) 

In [12]:
import_jobs(directory_to_import_to=directory_to_import_to, archive_directory=archive_directory, df=df)

In [13]:
pr = Project(directory_to_import_to)

In [14]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,13,finished,Al,murn,/murn,/home/jovyan/,new/murn,2020-11-10 11:14:21.689916,2020-11-10 11:14:45.113006,23.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1#11/11,Murnaghan,0.3.0,None,NaN
1,14,finished,Al,strain_0_9,/strain_0_9,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:22.912227,2020-11-10 11:14:23.365173,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
2,15,finished,Al,strain_0_92,/strain_0_92,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:24.927981,2020-11-10 11:14:25.394044,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
3,16,finished,Al,strain_0_94,/strain_0_94,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:26.931778,2020-11-10 11:14:27.421954,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
4,17,finished,Al,strain_0_96,/strain_0_96,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:29.007742,2020-11-10 11:14:29.431261,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
5,18,finished,Al,strain_0_98,/strain_0_98,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:30.930220,2020-11-10 11:14:31.406885,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
6,19,finished,Al,strain_1_0,/strain_1_0,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:32.937797,2020-11-10 11:14:33.441668,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
7,20,finished,Al,strain_1_02,/strain_1_02,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:34.940175,2020-11-10 11:14:35.461496,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
8,21,finished,Al,strain_1_04,/strain_1_04,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:37.038421,2020-11-10 11:14:37.536668,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
9,22,finished,Al,strain_1_06,/strain_1_06,/home/jovyan/,new/murn/murn_hdf5,2020-11-10 11:14:39.139254,2020-11-10 11:14:39.683094,0.0,pyiron@jupyter-jan-2djanssen-2dpyi-2dort-2dto-2dmybinder-2d2qja9tbo#1,Lammps,0.1,None,13.0
